In [80]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import requests
import numpy as np
import os
from dotenv import load_dotenv
from scipy.optimize import minimize, differential_evolution

load_dotenv()
px.set_mapbox_access_token(os.getenv("MAPBOX_TOKEN"))

In [81]:
# census tract svi ata
svi_df = pd.read_csv("https://svi.cdc.gov/Documents/Data/2020/csv/states/Maryland.csv")
#filter 
svi_df = svi_df[svi_df['STATE']=='Maryland'].reset_index()
# svi_df = svi_df[svi_df['COUNTY']=='Baltimore City'].reset_index()
# svi_df = svi_df[svi_df['COUNTY']=m

grocery_stores = pd.read_csv("https://opendata.arcgis.com/api/v3/datasets/85924b7086ef4506b4f2240d282a54c0_0/downloads/data?format=csv&spatialRefId=2248&where=1%3D1")
# lat lon
grocery_coords = list(zip(grocery_stores['latitude'], grocery_stores['longitude']))


In [82]:
# maryland boundary geojson
response = requests.get('https://opendata.arcgis.com/api/v3/datasets/2cb6e7dfc87047ccaa38f59d955d907b_2/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1')
tracts = json.loads(response.text)

filtered_features = [feature for feature in tracts["features"] if feature["properties"]["COUNTYFP20"] == '510']
tracts["features"] = filtered_features

In [ ]:
fig = px.choropleth_mapbox(svi_df, geojson=tracts, locations='FIPS', featureidkey='properties.GEOID20', color='RPL_THEMES', #EP_Noveh: veh percent,  RPL_THEMES: svi percentile
                           color_continuous_scale="matter",
                           range_color=(0, 1),
                        #    mapbox_style="dark",
                           zoom=10.5, center = {"lat": 39.29, "lon": -76.62},
                           opacity=0.85,
                           hover_data = ['COUNTY', "RPL_THEMES", "EP_NOVEH"],
                           labels={'RPL_THEMES':'SVI National Percentile', 'EP_NOVEH':'%Household without vehichles'},
                           width = 800,
                           height=800
                          )
# fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0}, title = "Baltimore City")
# fig = fig.update_traces(
#     marker_line_width=0.1
# )

groceryTrace = px.scatter_mapbox(
    data_frame=grocery_stores,
    opacity=1,
    hover_name = 'storename',
    lat='latitude',
    lon='longitude',
    text='address'
)
groceryTrace.update_traces(mode = "markers", marker = dict(color='#34eb64', size=10))

fig.add_trace(groceryTrace.data[0])

# fig.show()
fig.show()

In [84]:
import math
#thanks chatgpt
def haversine_distance(lat1, lon1, lat2, lon2):
    # distance between latitudes
    # and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0

    # apply formulae
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 3958.8
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

In [85]:
tracts_ID = [feature['properties']['GEOID20'] for feature in tracts['features']]
tracts_lat = pd.to_numeric([feature['properties']['INTPTLAT20'] for feature in tracts['features']])
tracts_lon = pd.to_numeric([feature['properties']['INTPTLON20'] for feature in tracts['features']])
tracts_locations = pd.DataFrame({'FIPS': tracts_ID, 'lat': tracts_lat, 'lon': tracts_lon})

In [100]:
def flattened_to_pair(flattened_list):
    return [(flattened_list[i], flattened_list[i+1]) for i in range(0, len(flattened_list), 2)] 

def dist_to_closest_grocery(lat, lon, grocery_coords):
    return min([haversine_distance(lat, lon, groc_lat,groc_lon) for groc_lat,groc_lon in grocery_coords])

def avg_dist_to_grocery(new_locations):
    new_locations = flattened_to_pair(new_locations)
    # create tuples from list
    new_grocery_coords = grocery_coords + new_locations
    new_dists = [dist_to_closest_grocery(lat, lon, new_grocery_coords) for lat, lon in zip(tracts_lat, tracts_lon)]
    return np.mean(new_dists)

In [ ]:
# tracts_locations['FIPS']= tracts_locations['FIPS'].astype(int)
tracts_locations["dist_to_groc"] = tracts_locations.apply(lambda x : dist_to_closest_grocery(x['lat'], x['lon'], grocery_coords), axis=1)
np.mean(tracts_locations["dist_to_groc"])

In [ ]:
print(avg_dist_to_grocery([]))

In [ ]:
# initial = (40, -76)

bounds = [(39, 39.37), (-76.7, -76.5)] * 3

# Optimization using the minimize function from scipy
# options = {'maxiter': 1000000}
# result = (avg_dist_to_grocery, initial, method='dogleg', options=options, tol=1e-16)

result = differential_evolution(avg_dist_to_grocery, bounds, disp=True, tol=1e-12)


print(result)


In [ ]:

result_df = pd.DataFrame(flattened_to_pair(result.x), columns=['lat', 'lon'])
result_df

In [ ]:
result_trace = px.scatter_mapbox(
    data_frame= result_df,
    opacity=1,
    lat='lat',
    lon='lon',
)
result_trace.update_traces(mode = "markers", marker = dict(color='red', size=12))

fig.add_trace(result_trace.data[0])

In [107]:
fig.write_html("./best_3_by_tracts.html")